# AQS Data Import

Simple notebook to import AQS data from CSV file.

In [ ]:
# Import pandas
import pandas as pd

# Load AQS data from CSV
df = pd.read_csv('../data/hourly_88101_2020.csv')

print(f"Data loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
#remove duplicates
df = df.drop_duplicates()
print(f"Shape after removing duplicates: {df.shape}")
print(f"Columns after removing duplicates: {len(df.columns)}") 

#Uncap display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Display first few rows
df.head()

In [ ]:
# get rows where sample value is greater than 100
df[df['Sample Measurement'] > 100]


In [ ]:
# Filter to essential columns
# Drop rows if 'Site Num' is not 10
print(df.columns.tolist())
df = df.drop(df[df["Site Num"] != 10].index)

df = df[['Date GMT', 'Time GMT', 'Sample Measurement']]

# Sort by date then time
df = df.sort_values(['Date GMT', 'Time GMT'])

# Reset index after sorting
df = df.reset_index(drop=True)

# Check the results
print(f"Shape: {df.shape}")
df.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
datetime_gmt = pd.to_datetime(df['Date GMT'] + ' ' + df['Time GMT'])
plt.plot(datetime_gmt, df['Sample Measurement'], marker='o', linestyle='-', markersize=2)
plt.title('Sample Measurement Over Time (GMT)')
plt.xlabel('Date and Time (GMT)')
plt.ylabel('Sample Measurement')
plt.grid()
plt.show()
 

In [ ]:
# Interactive plot with infinite zoom using Plotly
import plotly.express as px
import plotly.graph_objects as go

# Create datetime column
df['datetime_gmt'] = pd.to_datetime(df['Date GMT'] + ' ' + df['Time GMT'])

# Create interactive plot
fig = px.line(df, 
              x='datetime_gmt', 
              y='Sample Measurement',
              title='PM2.5 Measurements Over Time (Interactive)',
              labels={'datetime_gmt': 'Date and Time (GMT)', 
                     'Sample Measurement': 'PM2.5 (µg/m³)'},
              hover_data=['Date GMT', 'Time GMT'])

# Customize the plot
fig.update_traces(line=dict(width=1))
fig.update_layout(
    xaxis_title="Date and Time (GMT)",
    yaxis_title="PM2.5 (µg/m³)",
    hovermode='x unified'
)

fig.show()

In [ ]:
#we want to normalize the sample measurement column
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['Sample Measurement Normalized'] = scaler.fit_transform(df[['Sample Measurement']])

df.head()

In [ ]:
# Create datetime column
df['datetime_gmt'] = pd.to_datetime(df['Date GMT'] + ' ' + df['Time GMT'])

# Create interactive plot
fig = px.line(df, 
              x='datetime_gmt', 
              y='Sample Measurement Normalized',
              title='PM2.5 Measurements Over Time (Interactive)',
              labels={'datetime_gmt': 'Date and Time (GMT)', 
                     'Sample Measurement': 'PM2.5 (µg/m³)'},
              hover_data=['Date GMT', 'Time GMT'])

# Customize the plot
fig.update_traces(line=dict(width=1))
fig.update_layout(
    xaxis_title="Date and Time (GMT)",
    yaxis_title="PM2.5 (µg/m³)",
    hovermode='x unified'
)

fig.show()

In [ ]:
# Comprehensive AQS Data Analysis - Python/Pandas Version
print("\n=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Date range: {df['Date GMT'].min()} to {df['Date GMT'].max()}")

print("\n=== COLUMN ANALYSIS ===")
for col in df.columns:
    print(f"{col}: {df[col].dtype}, {df[col].nunique():,} unique values")

print("\n=== PM2.5 MEASUREMENT STATISTICS ===")
stats = df['Sample Measurement'].describe()
print(stats)
print(f"Missing values: {df['Sample Measurement'].isna().sum():,}")

# State analysis (if 'State Name' exists)
if 'State Name' in df.columns:
    state_stats = df.groupby('State Name').size().reset_index(name='Count')
    print("\n=== STATE ANALYSIS ===")
    print(state_stats.sort_values('Count', ascending=False).head(10))
else:
    print("\n=== STATE ANALYSIS ===")
    print("Column 'State Name' not found in df.")

# High pollution episodes (>35 µg/m³ - EPA standard)
high_pollution = df[df['Sample Measurement'] > 35]
print("\n=== HIGH POLLUTION EPISODES ===")
print(f"Records above 35 µg/m³: {len(high_pollution):,} ({len(high_pollution)/len(df)*100:.2f}%)")
if len(high_pollution) > 0:
    top_readings = high_pollution.nlargest(10, 'Sample Measurement')
    print("\nTop 10 highest readings:")
    print(top_readings[['Date GMT', 'Time GMT', 'Sample Measurement']])

# Site analysis (if 'Site Num' exists)
if 'Site Num' in df.columns:
    site_counts = df.groupby('Site Num').size().reset_index(name='Records')
    print("\n=== MONITORING SITE ANALYSIS ===")
    print(f"Total unique sites: {len(site_counts):,}")
    print("\nMost active sites:")
    print(site_counts.nlargest(10, 'Records'))
else:
    print("\n=== MONITORING SITE ANALYSIS ===")
    print("Column 'Site Num' not found in df.")

# Temporal patterns
if 'datetime_gmt' in df.columns:
    monthly_avg = df.groupby(df['datetime_gmt'].dt.to_period('M'))['Sample Measurement'].mean()
    hourly_avg = df.groupby(df['datetime_gmt'].dt.hour)['Sample Measurement'].mean()
    print("\n=== TEMPORAL PATTERNS ===")
    print("Monthly averages:")
    print(monthly_avg)
    print("\nHourly averages:")
    print(hourly_avg)
else:
    print("\n=== TEMPORAL PATTERNS ===")
    print("Column 'datetime_gmt' not found in df.")

print("\n=== DATA QUALITY ASSESSMENT ===")
print(f"Duplicate rows: {df.duplicated().sum():,}")
print(f"Negative measurements: {(df['Sample Measurement'] < 0).sum():,}")
print(f"Zero measurements: {(df['Sample Measurement'] == 0).sum():,}")
print(f"Extremely high measurements (>500): {(df['Sample Measurement'] > 500).sum():,}")

# Coordinate system analysis (if columns exist)
coord_cols = [c for c in df.columns if 'Latitude' in c or 'Longitude' in c or 'Datum' in c]
if coord_cols:
    coord_systems = df[coord_cols].drop_duplicates()
    print("\n=== COORDINATE SYSTEMS ===")
    print("Coordinate systems in use:")
    print(coord_systems)
else:
    print("\n=== COORDINATE SYSTEMS ===")
    print("No coordinate system columns found.")

print("\n=== ANALYSIS COMPLETE ===")
print("Variables available for further analysis:")
print("- df: Complete dataset")
if 'state_stats' in locals(): print("- state_stats: State-level statistics")
print("- high_pollution: Records above EPA standard")
if 'monthly_avg' in locals(): print("- monthly_avg: Monthly pollution averages")
if 'hourly_avg' in locals(): print("- hourly_avg: Hourly pollution averages")
if 'site_counts' in locals(): print("- site_counts: Site activity summary")


In [ ]:
# Connect to the SQLite database in the data folder and list all tables
from pathlib import Path
import sqlite3


db_path = Path('../data/aqs_data.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query to list all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in aqs_data.db:")
for table in tables:
    print(table[0])

cursor.close()
 

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

# Define database path
db_path = Path('../data/aqs_data.db')

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Example query: select first 5 rows from the main table
query = 'SELECT * FROM hourly_88101_2020 LIMIT 5'

# Load query results into a pandas DataFrame
df_demo = pd.read_sql_query(query, conn)

# Show the DataFrame
print(df_demo)

# Close the connection
conn.close()

In [ ]:
import sqlite3
from pathlib import Path
import pandas as pd

# Connect to database
db_path = Path('../data/aqs_data.db')
conn = sqlite3.connect(db_path)

# First, let's get basic info about the table WITHOUT loading all data
print("=== TABLE OVERVIEW ===")

# Get row count
count_query = "SELECT COUNT(*) FROM hourly_88101_2020"
total_rows = pd.read_sql_query(count_query, conn).iloc[0, 0]
print(f"Total rows in database: {total_rows:,}")

# Get column info
pragma_query = "PRAGMA table_info(hourly_88101_2020)"
columns_info = pd.read_sql_query(pragma_query, conn)
print(f"\nColumns ({len(columns_info)}):")
for _, row in columns_info.iterrows():
    print(f"  {row['name']} ({row['type']})")

# Get sample of data (first 1000 rows)
print("\n=== SAMPLE DATA (First 1000 rows) ===")
sample_query = """
SELECT * FROM hourly_88101_2020 
ORDER BY "Date GMT", "Time GMT"
LIMIT 1000
"""
df_sample = pd.read_sql_query(sample_query, conn)
print(f"Sample shape: {df_sample.shape}")
print(f"Date range in sample: {df_sample['Date GMT'].min()} to {df_sample['Date GMT'].max()}")

# # Show basic stats on Sample Measurement without loading all data
# print("\n=== SAMPLE MEASUREMENT STATISTICS ===")
# stats_query = """
# SELECT 
#     COUNT(*) as count,
#     AVG("Sample Measurement") as mean,
#     MIN("Sample Measurement") as min,
#     MAX("Sample Measurement") as max,
#     COUNT(CASE WHEN "Sample Measurement" > 35 THEN 1 END) as above_epa_standard
# FROM hourly_88101_2020
# """
# stats = pd.read_sql_query(stats_query, conn)
# print(stats)

# Display sample data
print("\nFirst 10 rows:")
df_sample.head(10)

In [ ]:
# Smart querying approach for large datasets
import sqlite3
import pandas as pd
from pathlib import Path

# Connect to database
db_path = Path('../data/aqs_data.db')
conn = sqlite3.connect(db_path)

# 1. RANDOM SAMPLE for exploration (fast)
print("=== RANDOM SAMPLE (10,000 records) ===")
sample_query = """
SELECT * FROM hourly_88101_2020 
ORDER BY RANDOM() 
LIMIT 10000
"""
df_sample = pd.read_sql_query(sample_query, conn)
print(f"Sample shape: {df_sample.shape}")

# 2. SPECIFIC TIME PERIOD (e.g., one month)
print("\n=== JANUARY 2020 DATA ===")
january_query = """
SELECT * FROM hourly_88101_2020 
WHERE "Date GMT" BETWEEN '2020-01-01' AND '2020-01-31'
"""
df_january = pd.read_sql_query(january_query, conn)
print(f"January shape: {df_january.shape}")

# 3. SPECIFIC STATES (e.g., California only)
print("\n=== CALIFORNIA DATA ===")
california_query = """
SELECT * FROM hourly_88101_2020 
WHERE "State Name" = 'California'
"""
df_california = pd.read_sql_query(california_query, conn)
print(f"California shape: {df_california.shape}")

# # 4. HIGH POLLUTION EVENTS ONLY
# print("\n=== HIGH POLLUTION EVENTS ===")
# high_pollution_query = """
# SELECT * FROM hourly_88101_2020 
# WHERE "Sample Measurement" > 35
# ORDER BY "Sample Measurement" DESC
# """
# df_high_pollution = pd.read_sql_query(high_pollution_query, conn)
# print(f"High pollution events: {df_high_pollution.shape}")

# # 5. AGGREGATED DATA (fast server-side processing)
# print("\n=== STATE-LEVEL AGGREGATIONS ===")
# state_agg_query = """
# SELECT 
#     "State Name",
#     COUNT(*) as total_measurements,
#     AVG("Sample Measurement") as avg_pm25,
#     MIN("Sample Measurement") as min_pm25,
#     MAX("Sample Measurement") as max_pm25,
#     COUNT(CASE WHEN "Sample Measurement" > 35 THEN 1 END) as high_pollution_days
# FROM hourly_88101_2020
# GROUP BY "State Name"
# ORDER BY avg_pm25 DESC
# """
# df_state_stats = pd.read_sql_query(state_agg_query, conn)
# print(f"State aggregations: {df_state_stats.shape}")
# print(df_state_stats.head())

# Now you have manageable DataFrames for analysis:
print("\n=== AVAILABLE DATAFRAMES ===")
print(f"df_sample: {df_sample.shape} - Random sample for exploration")
print(f"df_january: {df_january.shape} - January 2020 data")
print(f"df_california: {df_california.shape} - California data")
# print(f"df_high_pollution: {df_high_pollution.shape} - High pollution events")
# print(f"df_state_stats: {df_state_stats.shape} - State-level statistics")

conn.close()

In [ ]:
# df for california, only site num 0019. Ordered by Date GMT and Time GMT
df_ca_0019 = df_california[(df_california['Site Num'] == '0019')]
df_ca_0019 = df_ca_0019.sort_values(['Date GMT', 'Time GMT']).reset_index(drop=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df_ca_0019.head()

In [ ]:
# Convert Sample Measurement to numeric first, then get value counts ordered by value
df_california['Sample Measurement'] = pd.to_numeric(df_california['Sample Measurement'])
df_california['Sample Measurement'].value_counts().sort_index()

 

In [ ]:
df_california[df_california['Sample Measurement'] < -10]

In [ ]:
# Count distinct 'Uncertainty' values and show the count for each value
uncertainty_counts = df_california['Uncertainty'].value_counts().sort_index()
print(uncertainty_counts)

In [ ]:
#df where uncertainty is not null
df_uncertainty = df_california[df_california['Uncertainty'].notnull()] 
#print uncertainty order by uncertainty
df_uncertainty = df_uncertainty.sort_values('Uncertainty').reset_index(drop=True)
df_uncertainty.tail(100)
 

In [ ]:
#plot uncertainty histogram
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.hist(df_california['Uncertainty'].dropna(), bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Uncertainty in California PM2.5 Measurements')
plt.xlabel('Uncertainty')
plt.ylabel('Frequency')
plt.grid()
plt.show()  

In [ ]:
#plot uncertainty histogram for 
import matplotlib.pyplot as plt
plt.figure(figsize=(100, 60))
plt.hist(df_california['Uncertainty'].dropna(), bins=500, color='skyblue', edgecolor='black')
plt.title('Distribution of Uncertainty in California PM2.5 Measurements')
plt.xlabel('Uncertainty')
plt.ylabel('Frequency')
plt.grid()
plt.show() 

In [ ]:

import matplotlib.pyplot as plt

# Plot histograms and boxplots for all columns in df_california
for col in df_california.columns:
    plt.figure(figsize=(12, 5))
    # Try to convert to numeric for plotting, otherwise skip non-numeric columns
    try:
        data = pd.to_numeric(df_california[col], errors='coerce')
        if data.notnull().sum() > 0:
            plt.subplot(1, 2, 1)
            plt.hist(data.dropna(), bins=500, color='skyblue', edgecolor='black')
            plt.title(f'Histogram of {col}')
            plt.xlabel(col)
            plt.ylabel('Frequency')
            plt.grid()

            plt.subplot(1, 2, 2)
            plt.boxplot(data.dropna(), vert=False)
            plt.title(f'Boxplot of {col}')
            plt.xlabel(col)
            plt.grid()
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"Skipping column {col}: {e}")